In [1]:
#main
import numpy as np
import pandas as pd
import pandas_profiling
import time
import datetime

# support
import corelib

# csv to pd loader
from corelib.data_loader import loader

# reduce memory usage for pd-index
from corelib.data_prep import reduce_mem_usage
from corelib.data_prep import reduce_obj_mem_usage

# searching for unical ordered values
from corelib.data_prep import search_func

#dump data with
from corelib.data_dump import dumper

С какими задачами можно столкнуться и зачем вообще питонисту работать с данными?

In [6]:
data = loader(mode='extract', index_col='installation_id')
data.keys()

dict_keys(['train'])

In [7]:
df_train = data['train']
del data

Почему данные нужно приводить к какому-то виду?
Что за проблема вычислительной сложности?
pandas и numpy
Какие данные в pandas весят больше всего и почему

In [9]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)

In [10]:
pandas_profiling.ProfileReport(df_train)

C:\Users\Konstantin.mne\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,11
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,88.0 B
Numeric,3
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [11]:
df_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 0001e90f to 04a5927d
Data columns (total 10 columns):
event_id        200000 non-null object
game_session    200000 non-null object
timestamp       200000 non-null object
event_data      200000 non-null object
event_count     200000 non-null int64
event_code      200000 non-null int64
game_time       200000 non-null int64
title           200000 non-null object
type            200000 non-null object
world           200000 non-null object
dtypes: int64(3), object(7)
memory usage: 147.3 MB


In [12]:
# reduce memory usage
df_train = reduce_mem_usage(df_train)

Mem. usage decreased to 13.73 Mb (18.2% reduction)


In [13]:
df_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 0001e90f to 04a5927d
Data columns (total 10 columns):
event_id        200000 non-null object
game_session    200000 non-null object
timestamp       200000 non-null object
event_data      200000 non-null object
event_count     200000 non-null int16
event_code      200000 non-null int16
game_time       200000 non-null int32
title           200000 non-null object
type            200000 non-null object
world           200000 non-null object
dtypes: int16(2), int32(1), object(7)
memory usage: 144.3 MB


In [ ]:
# распаковываем json

In [ ]:
# обычный и дип.мемори

In [ ]:
# смотрим данные, опять что-то не так - недораспакованные строки, флоты, фокусы с типами (None и були), разреженность

In [ ]:
# разбираем залежи нераспакованных вложений

In [ ]:
# объекты и категориальные типы - соращаем тут

In [ ]:
# замена пропусков

In [ ]:
# сериализация - нюансы

In [ ]:
# перегоняем в numpy и пример расчета модели, сравнение